In [75]:
#http://docs.opentrons.com/robot.html#simulate
#robot.run()
#vim /usr/local/lib/python3.4/dist-packages/opentrons/__init__.py version change from 3.5 to 3.4

robot.reset()
#print(robot.get_containers())
print(robot.get_instruments())
print(robot.commands())
robot._driver.reset()
#robot.disconnect()


[]
[]


RuntimeError: No connection found.

In [76]:
from opentrons import robot
robot.connect('Virtual Smoothie')
#robot.connect(['COM3'])
#connect_to_robot()

print('Connected, homing...')
robot.home()
print(robot.get_instruments())
print(robot.commands())
                    

Connected, homing...
[]
[]


In [77]:
from opentrons import containers, instruments

p200rack = containers.load('tiprack-200ul', 'A1', 'tips')
plate = containers.load('96-PCR-flat', 'C1', 'plat')
trough = containers.load('tube-rack-.75ml', 'C3', 'trough')
trash = containers.load('tiprack-200ul', 'A1', 'trash')

p200 = instruments.Pipette(
    name="p200",
    trash_container=trash,
    tip_racks=[p200rack],
    max_volume=200,
    axis="b"
)



In [82]:

fspot='E1'
p200.pick_up_tip(p200rack['A2'])
p200.aspirate(175, trough['A1']) 
p200.dispense(100, plate[fspot])
p200.drop_tip(p200rack['A2'])

p200.pick_up_tip(p200rack['A3'])
p200.aspirate(125, trough['B1']) 
p200.dispense(50, plate[fspot])
p200.drop_tip(p200rack['A3'])

p200.pick_up_tip(p200rack['A4'])
p200.aspirate(75, trough['C1'])  
p200.dispense(25, plate[fspot])

for i in range(10):
    p200.aspirate(100, plate[fspot]) 
    p200.dispense(100, plate[fspot])


p200.drop_tip(p200rack['A4'])
for warning in robot.simulate():
    print(warning)




In [83]:
print("\n".join(robot.commands()))

Picking up tip from <Deck>/<Slot A1>/<Container tips>/<Well A2>
Aspirating 175uL at <Deck>/<Slot C3>/<Container trough>/<Well A1>
Dispensing 100uL at <Deck>/<Slot C1>/<Container plat>/<Well E1>
Drop_tip at <Deck>/<Slot A1>/<Container tips>/<Well A2>
Picking up tip from <Deck>/<Slot A1>/<Container tips>/<Well A3>
Aspirating 125uL at <Deck>/<Slot C3>/<Container trough>/<Well B1>
Dispensing 50uL at <Deck>/<Slot C1>/<Container plat>/<Well E1>
Drop_tip at <Deck>/<Slot A1>/<Container tips>/<Well A3>
Picking up tip from <Deck>/<Slot A1>/<Container tips>/<Well A4>
Aspirating 75uL at <Deck>/<Slot C3>/<Container trough>/<Well C1>
Dispensing 25uL at <Deck>/<Slot C1>/<Container plat>/<Well E1>
Aspirating 100uL at <Deck>/<Slot C1>/<Container plat>/<Well E1>
Dispensing 100uL at <Deck>/<Slot C1>/<Container plat>/<Well E1>
Drop_tip at <Deck>/<Slot A1>/<Container tips>/<Well A4>
Picking up tip from <Deck>/<Slot A1>/<Container tips>/<Well A2>
Aspirating 175uL at <Deck>/<Slot C3>/<Container trough>/<Well

In [69]:
robot.reset()

Connected, homing...


[]

In [17]:
robot.move_to(plate[0])

In [26]:
def connect_to_robot():
    """
    Search for ports, connect to a robot, and home
    """
    print('Enter the number of the port to connect to:\n')
    count = 0
    ports = robot.get_serial_ports_list()
    if not ports:
        raise RuntimeError(
            'No serial ports found. Please make sure your robot\'s USB cable is connected')
    for i, portname in enumerate(ports):
        print('  {0})   {1}\n'.format(i, portname))
    val = input('Enter number next to desired port (0, 1, etc): '.format(1, count))
    try:
        val = int(val)
    except:
        print('Bad input: {}'.format(val))
        sys.exit()
    try:
        robot.connect(robot.get_serial_ports_list()[val])
    except RuntimeWarning:
        robot.connect(robot.get_serial_ports_list()[val])
    print('Connected, homing...')
    robot.home()
    robot.home('xy')


def request_distance_from_user():
    """
    Get the dimensions of this robot, and ask the user for the testing distance
    """
    dimensions = robot._driver.get_dimensions() - (50, 50, 0)
    x_dist = int(input('Enter distanct for X to travel (max {}):'.format(dimensions[0])))
    y_dist = int(input('Enter distanct for Y to travel (max {}):'.format(dimensions[1])))
    x_dist = min(x_dist, dimensions[0])
    y_dist = min(y_dist, dimensions[1])
    return (x_dist, y_dist)

    
def measure_belts(x_dist, y_dist):
    """
    Move the robot, allow user to mark positions, ask user to measure and submit measurements
    """
    print('Moving the robot {0}mm on the X, and {1}mm on the Y'.format(x_dist, y_dist))
    robot.move_head(x=x_dist, y=y_dist * -1, mode='relative')  # Y axis is flipped

    input('mark X position on the robot, then press ENTER key')
    input('mark Y position on the robot, then press ENTER key')

    robot.home('xy')
    robot.home('xy')

    actual_travel_x = float(input('enter the measured X distance: '))
    actual_travel_y = float(input('enter the measured Y distance: '))
    robot._driver.calibrate_steps_per_mm('x', x_dist, actual_travel_x)
    robot._driver.calibrate_steps_per_mm('y', y_dist, actual_travel_y)


connect_to_robot()
x, y = request_distance_from_user()
measure_belts(x, y)

print('New values saved. Restarting the robot to allow changes to take effect')
print('Please wait 5-10 seconds before reconnecting to the robot')

robot._driver.reset()
robot.disconnect()

Enter the number of the port to connect to:



RuntimeError: No serial ports found. Please make sure your robot's USB cable is connected

['Picking up tip from <Deck>/<Slot A1>/<Container tips>/<Well A2>', 'Aspirating 175uL at <Deck>/<Slot C3>/<Container trough>/<Well A1>', 'Dispensing 100uL at <Deck>/<Slot C1>/<Container plat>/<Well E10>', 'Dispensing 50uL at <Deck>/<Slot C1>/<Container plat>/<Well E11>', 'Dispensing 25uL at <Deck>/<Slot C1>/<Container plat>/<Well E12>', 'Drop_tip at <Deck>/<Slot A1>/<Container tips>/<Well A2>', 'Picking up tip from <Deck>/<Slot A1>/<Container tips>/<Well A3>', 'Aspirating 125uL at <Deck>/<Slot C3>/<Container trough>/<Well B1>', 'Dispensing 50uL at <Deck>/<Slot C1>/<Container plat>/<Well E11>', 'Dispensing 75uL at <Deck>/<Slot C1>/<Container plat>/<Well E12>', 'Drop_tip at <Deck>/<Slot A1>/<Container tips>/<Well A3>', 'Picking up tip from <Deck>/<Slot A1>/<Container tips>/<Well A4>', 'Aspirating 75uL at <Deck>/<Slot C3>/<Container trough>/<Well C1>', 'Dispensing 25uL at <Deck>/<Slot C1>/<Container plat>/<Well E10>', 'Dispensing 25uL at <Deck>/<Slot C1>/<Container plat>/<Well E11>', 'Dis

In [2]:
!pip install --upgrade opentrons

  Using cached opentrons-2.2.0-py2.py3-none-any.whl
  Using cached pyserial-3.2.1-py2.py3-none-any.whl
  Running setup.py install for pyreadline: started
    Running setup.py install for pyreadline: finished with status 'done'
  Running setup.py install for dill: started
    Running setup.py install for dill: finished with status 'done'
